<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MISTRAL_TUTORIAL_T4GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MISTRAL TUTORIAL

# Run the DEMO - Run the model from the Source

https://github.com/mistralai/mistral-src/tree/main

In [ ]:
from pathlib import Path

code_path = "/content/mistral-src/"  # codebase
data_path = Path("/content/gdrive/MyDrive/datasets/mistral/mistral-7B-v0.1/Symptom2Disease.csv")  # dataset downloaded from Kaggle
model_path = Path("/content/gdrive/MyDrive/datasets/mistral/mistral-7B-v0.1/")  # model and tokenizer location

#!pip install -q -U mistral-src

!pip install colab-env --upgrade
import colab_env

!git clone https://github.com/mistralai/mistral-src.git
%cd /content/mistral-src
!pip install -r /content/mistral-src/requirements.txt

In [2]:
#### CLI FOR THE DEMO ########


#!python -m main demo /content/gdrive/MyDrive/datasets/mistral/mistral-7B-v0.1/

# To give your own prompts
#!python -m main interactive /content/gdrive/MyDrive/datasets/mistral/mistral-7B-v0.1/

#!python -m test_generate

#!python -m main interactive /content/gdrive/MyDrive/datasets/mistral/mistral-7B-v0.1/ --max_tokens 256 --temperature 1.0

In [3]:
#Imports
import csv
import tqdm
import torch
import numpy as np
from pathlib import Path

import torch
from torch import bfloat16

#model = Transformer.from_folder(model_path, dtype=bfloat16)

code_path = "/content/mistral-src/"  # codebase
data_path = Path("/content/gdrive/MyDrive/datasets/mistral/mistral-7B-v0.1/Symptom2Disease.csv")  # dataset downloaded from Kaggle
model_path = Path("/content/gdrive/MyDrive/datasets/mistral/mistral-7B-v0.1/")  # model and tokenizer location

import sys
sys.path.append(code_path)  # append the path where mistral-src was cloned

from mistral.model import Transformer
from mistral.tokenizer import Tokenizer
from main import generate

#Load the model and tokenizer # ALTERNATIVE

## T4 GPU
model0 = Transformer.from_folder(model_path, max_batch_size=3)

### A100 GPU
#model = Transformer.from_folder(model_path, dtype=torch.bfloat16)

tokenizer0 = Tokenizer(str(model_path / "tokenizer.model"))

max_tokens = 256
temperature = 1.0
prompt='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.'
response=generate([prompt], model0, tokenizer0, max_tokens=max_tokens, temperature=temperature)


#IF NOT A100 GPU
#'xformers.ops.fmha.attn_bias.BlockDiagonalCausalLocalAttentionMask'>
#    bf16 is only supported on A100+ GPUs
#    unsupported embed per head: 128


#tokenizer.batch_decode(response)[0]

#print(response[0])




In [12]:
max_tokens = 2024 # FOR A100 GPU FOR T4 WITH HIGH RAM max_tokens<=1024.
temperature = 0.9

prompt = "As a data scientist, can you explain the concept of regularization in machine learning? please the anser in english"

#prompt='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering in english.'

#prompt='Which country has the most natural lakes? Answer with only the country name.'

response=generate([prompt], model0, tokenizer0, max_tokens=max_tokens, temperature=temperature)
#print(response[0])
tokenizer0.decode(response[0])


#model = Transformer(args).to("cuda", dtype=torch.float32)
#tokenizer = DebugTokenizer()


'As a data scientist, can you explain the concept of regularization in machine learning? please the anser in english\n\n### Answers\n\nThe term regularization is used in a number of contexts in machine learning and statistics. It is most commonly used to refer to a technique for preventing overfitting in model estimation, by penalizing the complexity of the model. This can be achieved by adding a term to the objective function that penalizes the magnitude of the model coefficients or parameters, such as the L1 or L2 norm of the coefficients. Regularization can also be used to prevent overfitting in other ways, such as by constraining the model to be sparse or by using dropout during training.\n\nAnother common use of the term regularization is in the context of Bayesian inference, where it refers to the process of adding prior knowledge or assumptions to the model to help inform the estimation of the model parameters. This can be done by specifying a prior distribution for the model pa

# Dependencies

In [ ]:

!pip install langchain --quiet
!pip install accelerate --quiet
!pip install transformers --quiet
%pip install openai==0.28  --root-user-action=ignore
%pip install tiktoken
%pip install colab-env --upgrade --quiet --root-user-action=ignore
!pip install bitsandbytes --quiet

# LLM Configuration - Creation of the model and tokenizer - HuggingFace



In [6]:
#!pip install accelerate

import torch
from textwrap import fill
from IPython.display import Markdown, display

import colab_env
import os

device = "cuda"

access_token = os.getenv("HF_TOKEN")

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    )

from langchain import PromptTemplate
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader, UnstructuredURLLoader
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import warnings
warnings.filterwarnings('ignore')

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


#model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
#tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

#from transformers import AutoTokenizer, MistralForCausalLM

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [7]:
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import warnings
warnings.filterwarnings('ignore')

MODEL_NAME='mistralai/Mistral-7B-Instruct-v0.1'
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 512
generation_config.temperature = 0.9
generation_config.top_p = 0.9
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

#model.to(device)

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    pad_token_id=tokenizer.eos_token_id
)

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipeline)

# Examples

In [13]:
query='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.'
query1 = "who is the President of the USA?"
query2 = "Who won the baseball World Series in 2020? and Who Lost"

device="cuda"
def prompt_completion(query):
    messages = [
        {"role": "user", "content": "%s"%query}
    ]

    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = encodeds.to(device)

    #https://stackoverflow.com/questions/69609401/suppress-huggingface-logging-warning-setting-pad-token-id-to-eos-token-id

    generated_ids = model.generate(model_inputs, max_new_tokens=512, do_sample=True, negative_prompt_attention_mask='attention_mask',
                    pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.batch_decode(generated_ids)
    print()
    print()
    result=decoded[0].replace('<s> [INST] %s [/INST]'%query,"")
    result=result.replace('</s>',"")
    print('Prompt: %s'%query)
    print('-'*80)
    print('Answer: %s'%result)

prompt_completion(query)
print()
print('='*80)
prompt_completion(query1)
print()
print('='*80)
prompt_completion(query2)

query3 = "what is the 20.5% of 40?"
query4 = "As a data scientist, can you explain the concept of regularization in machine learning?"
query5 ='Which country has the most natural lakes? Answer with only the country name.'

print()
print('='*80)
prompt_completion(query3)
print()
print('='*80)
prompt_completion(query4)
print()
print('='*80)
prompt_completion(query5)


query6 = "How AWS has evolved?"
print()
print('='*80)
prompt_completion(query6)




Prompt: I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.
--------------------------------------------------------------------------------
Answer:  When you bought an ice cream for 6 kids, you spent $7.50. This is calculated as:

6 x $1.25 = $7.50

You spent $7.50 on the ice cream.

The next part of the question is whether you got any money back, which is different from the first part. You paid for the ice cream with a $10 bill, but you only spent $7.50, so you got $2.50 back. This is calculated as:

$10 - $7.50 = $2.50

You got $2.50 back from the ice cream. So the final answer is that you got $2.50 back, but you spent $7.50 on the ice cream.



Prompt: who is the President of the USA?
--------------------------------------------------------------------------------
Answer:  Currently, Joe Biden is the President of the United States of America. He was inaugurated on January 20, 2021.



P

In [10]:
prompt="summarize the following text: AI analytics is a subset of business intelligence that uses machine learning algorithms to monitor and analyze vast amounts of data constantly,\
 automating the time-consuming work generally done by a data analyst[1]. AI analytics results from automating business analytics processes through artificial intelligence (AI) and machine learning (ML)[2]. \
 AI analytics can improve the scale and speed at which data is analyzed compared to a human worker. \
 It can identify trends, provide insights, and discover relationships within data [2]. \
 AI analytics uses machine learning algorithms to monitor and analyze vast amounts of data, automating the time-consuming work of a data analyst[1].\
  AI analytics extracts valuable information from big data using advanced machine learning (ML) and AI tasks like natural language processing (NLP). \
  It is used to identify patterns and trends in data for informed decision-making[3]. \
Here are some examples of AI analytics: \
Sentiment analysis examines information about a subject online and evaluates its consumer feedback [4].\
Predictive analytics and forecasting: AI analysis tools can analyze historical data, market data, and other factors to forecast \
future sales and predict purchasing patterns [4-5].\
Anomaly detection and fraud prevention: AI analytics can detect anomalies in data that may indicate fraudulent activity[4].\
Image and video analysis: AI analytics can analyze images and videos to identify objects, people, and other features[4].\
Conversation analytics: AI analytics can analyze conversations between customers and customer service representatives to identify trends \
and improve customer service[6].\
Synthetic data creation: AI analytics can be used to create synthetic data that can be used to train machine learning models[4]. \
Dashboard creation: AI analytics can create dashboards that provide real-time insights into business performance[7].\
Data entry automation: AI analytics can be used to automate data entry tasks, freeing up time for more critical tasks[7]. \
You can check out [5-7] for more examples. \
Some of the challenges of AI analytics include:\
Repeatability and reproducibility: One of the biggest challenges for AI and data analytics in a production environment is the issue of \
repeatability and reproducibility[8]. \
Speed of implementation: Implementing AI and data analytics can be time-consuming and take a long time to see results[8].\
Using the correct data: AI and data analytics require large amounts of data to be effective, but using the correct data is essential[8]. \
Deploying models at scale: Deploying AI and data analytics models at scale can be challenging, requiring significant computing power and \
infrastructure[8]. \
Managing the complexity of AI and data analytics: AI and data analytics can be complex, and it can be challenging to understand how they work \
and how to use them effectively [8]. You can check out [8-10] for more information.\
Some benefits of AI analytics include: \
Real-time insights: AI analytics can provide real-time insights into business performance, \
allowing companies to make informed decisions quickly [11]. \
Improved decision-making: AI analytics can help companies make better decisions by providing \
insights into customer behaviour, market trends, and other factors[12].\
Increased productivity and efficiency: AI analytics can automate tasks that would otherwise require human intervention, \
freeing up time for more critical tasks[12]. \
Cost savings through automation: AI analytics can help companies save money by automating tasks that would otherwise require human intervention[12].\
Predictive analytics and forecasting: AI analysis tools can analyze historical data, market data, and other factors to forecast future sales and \
predict purchasing patterns [11,13].\
Anomaly detection and fraud prevention: AI analytics can detect anomalies in data that may indicate fraudulent activity[11].\
Image and video analysis: AI analytics can analyze images and videos to identify objects, people, and other features[11]. \
Conversation analytics: AI analytics can analyze conversations between customers and customer service representatives to identify trends and \
improve customer service[11].\
Synthetic data creation: AI analytics can be used to create synthetic data that can be used to train machine learning models[12].\
Dashboard creation: AI analytics can create dashboards that provide real-time insights into business performance[12].\
Data entry automation: AI analytics can be used to automate data entry tasks, freeing up time for more critical tasks[. \
You can check out [11-14] for more information.\
AI Analytics and LLMs \
LLMs, or Large Language Models, are machine learning algorithms designed to understand and generate human-like text based on the data they have been\
trained on[15]. Traditional data analysis techniques primarily focus on structured and numerical data, but LLMs provide a valuable means of \
understanding and leveraging today's vast textual data. Pecan AI combines LLMs and predictive AI to simplify data analytics and enhance the \
user experience[15]. \
AI Analytics using LLMs is a significant advancement in data science and analytics. \
LLMs, or Large Language Models, are machine learning algorithms designed to understand and generate human-like text based on the data they've \
been trained on. LLMs work by predicting the likelihood of a sequence of words appearing in a sentence or a paragraph. This makes them exceptional \
tools for text generation, translation, and interpretation[15]. LLMs are not well-suited for making predictions on numerical data. However, \
LLMs are invaluable in data analysis and manipulation. LLMs provide a means to understand complex textual data, \
thus enabling the data to be used for further analysis and decision-making processes. LLMs offer unique capabilities that can significantly \
enhance the effectiveness of data analysis and decision-making processes1. Large language models excel at analyzing and processing textual data, \
enabling businesses to extract valuable insights from unstructured information[16]. Traditional data analysis techniques primarily focus on \
structured and numerical data, but LLMs provide a valuable means of understanding and leveraging today vast textual data. \
Pecan AI combines LLMs and predictive AI to simplify data analytics and \
enhance the user experience[16]."

print()
print('='*80)
prompt_completion(prompt)





Prompt: summarize the following text: AI analytics is a subset of business intelligence that uses machine learning algorithms to monitor and analyze vast amounts of data constantly, automating the time-consuming work generally done by a data analyst[1]. AI analytics results from automating business analytics processes through artificial intelligence (AI) and machine learning (ML)[2].  AI analytics can improve the scale and speed at which data is analyzed compared to a human worker.  It can identify trends, provide insights, and discover relationships within data [2].  AI analytics uses machine learning algorithms to monitor and analyze vast amounts of data, automating the time-consuming work of a data analyst[1].  AI analytics extracts valuable information from big data using advanced machine learning (ML) and AI tasks like natural language processing (NLP).   It is used to identify patterns and trends in data for informed decision-making[3]. Here are some examples of AI analytics: 